<a href="https://colab.research.google.com/github/dlehdgus99/machine_learning_projects/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import tensorflow as tf
import tensorflow.keras as keras 
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import os 
import time 
import matplotlib.pyplot as plt
import matplotlib.image as img
import pickle
import time

# %tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Found GPU at: /device:GPU:0
Mounted at /content/drive


In [139]:

# Size vector to generate images from
EPOCHS = 150
LEARNING_RATE = 0.0002
BETA=0.5
IMAGESIZE=64
BATCH_SIZE=16
DATA_PATH = 'drive/My Drive/dog_GAN/data'

In [140]:
dataset = keras.preprocessing.image_dataset_from_directory(
    directory=DATA_PATH, label_mode=None, image_size=(64,64), batch_size=BATCH_SIZE,
    shuffle=True
).map(lambda x: x/255.0)

Found 4739 files belonging to 1 classes.


In [141]:
latent_dim=64

generator=keras.Sequential(
    [
        keras.layers.Input(shape=(latent_dim,)),
        keras.layers.Dense(16*16*64),
        keras.layers.Reshape((16, 16, 64)),
        keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        keras.layers.BatchNormalization(momentum=0.8),
        keras.layers.ReLU(),
        keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        keras.layers.BatchNormalization(momentum=0.8),
        keras.layers.ReLU(),
        keras.layers.Conv2DTranspose(3, kernel_size=5, padding="same", activation="sigmoid"),
    ]

)
generator.summary()






Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 16384)             1064960   
                                                                 
 reshape_25 (Reshape)        (None, 16, 16, 64)        0         
                                                                 
 conv2d_transpose_89 (Conv2D  (None, 32, 32, 64)       65600     
 Transpose)                                                      
                                                                 
 batch_normalization_127 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 re_lu_61 (ReLU)             (None, 32, 32, 64)        0         
                                                                 
 conv2d_transpose_90 (Conv2D  (None, 64, 64, 64)     

In [142]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        keras.layers.BatchNormalization(momentum=0.8),
        keras.layers.LeakyReLU(alpha=0.2),
        keras.layers.Dropout(0.25),
        keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        keras.layers.BatchNormalization(momentum=0.8),
        keras.layers.LeakyReLU(alpha=0.2),
        keras.layers.Dropout(0.25),
        keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        keras.layers.BatchNormalization(momentum=0.8),
        keras.layers.LeakyReLU(alpha=0.2),
        keras.layers.Flatten(),
        keras.layers.Dense(1, activation="sigmoid")
    ])

discriminator.summary()






Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_63 (Conv2D)          (None, 32, 32, 64)        3136      
                                                                 
 batch_normalization_129 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 leaky_re_lu_66 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 dropout_58 (Dropout)        (None, 32, 32, 64)        0         
                                                                 
 conv2d_64 (Conv2D)          (None, 16, 16, 128)       131200    
                                                                 
 batch_normalization_130 (Ba  (None, 16, 16, 128)      512       
 tchNormalization)                                   

In [143]:
load_weights=False

if load_weights==True:
  generator.load_weights('gen_weights.h5')
  discriminator.load_weights('disc_weights.h5')

opt_gen = keras.optimizers.Adam(learning_rate=0.0002)
opt_disc = keras.optimizers.Adam(learning_rate=0.0002)
loss_fn = keras.losses.BinaryCrossentropy()

for epoch in range(EPOCHS):
  for idx, real in enumerate(tqdm(dataset)):
    batch_size=real.shape[0]
    if batch_size!=BATCH_SIZE:
      continue
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:

      
      fake = generator(random_latent_vectors)

      real_output= discriminator(real)
      fake_output= discriminator(fake)

      
      loss_disc_real = loss_fn(tf.ones((batch_size, 1)), real_output)
      loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), fake_output)

      loss_disc = (loss_disc_real + loss_disc_fake) / 2 
      loss_gen = loss_fn(tf.ones(batch_size, 1), fake_output)


      gen_grads=gen_tape.gradient(loss_gen,generator.trainable_weights)
      disc_grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
      

      opt_gen.apply_gradients(zip(gen_grads,generator.trainable_weights))
      opt_disc.apply_gradients(zip(disc_grads, discriminator.trainable_weights))
      


  print (f'Epoch {epoch+1}, gen loss={loss_gen},disc loss={loss_disc}')

  # save Image 
  img = keras.preprocessing.image.array_to_img(fake[0])
  img.save(f"drive/My Drive/dog_GAN/output/{epoch+1}.png")

  0%|          | 0/297 [00:00<?, ?it/s]

Epoch 1, gen loss=1.811697244644165,disc loss=0.31823956966400146


  0%|          | 0/297 [00:00<?, ?it/s]

Epoch 2, gen loss=4.594020366668701,disc loss=0.02837308868765831


  0%|          | 0/297 [00:00<?, ?it/s]

...

Epoch 148, gen loss=1.2559236288070679,disc loss=0.5833055377006531


  0%|          | 0/297 [00:00<?, ?it/s]

Epoch 149, gen loss=0.7677255272865295,disc loss=0.7321449518203735


  0%|          | 0/297 [00:00<?, ?it/s]

Epoch 150, gen loss=0.7714296579360962,disc loss=0.6653990149497986


In [144]:
generator.save_weights('gen_weights.h5')
discriminator.save_weights('disc_weights.h5')
